In [1]:
# !pip3 install idiomatch
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip install jieba
!pip install nltk
!pip install transformers
!pip install sentencepiece
!pip install -U synonyms
!python -c "import synonyms"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-04 09:04:23.500524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 09:04:24.637603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 92.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/p

In [2]:
import spacy
# from idiomatch import Idiomatcher
nlp_en = spacy.load("en_core_web_sm")
import synonyms
from jieba import posseg
# idiomatcher = Idiomatcher.from_pretrained(nlp_en)

Building prefix dict from /usr/local/lib/python3.10/dist-packages/synonyms/data/vocab.txt ...
DEBUG:jieba:Building prefix dict from /usr/local/lib/python3.10/dist-packages/synonyms/data/vocab.txt ...
Loading model from cache /tmp/jieba.uc3c121179eb864b10e70ae5fa25047be.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.uc3c121179eb864b10e70ae5fa25047be.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [/usr/local/lib/python3.10/dist-packages/synonyms/data/vocab.txt] ... 


Loading model cost 1.605 seconds.
DEBUG:jieba:Loading model cost 1.605 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


>> Synonyms on loading stopwords [/usr/local/lib/python3.10/dist-packages/synonyms/data/stopwords.txt] ...
>> Synonyms on loading vectors [/usr/local/lib/python3.10/dist-packages/synonyms/data/words.vector.gz] ...


In [3]:
from nltk.corpus import wordnet 
import jieba
import nltk
nltk.download('wordnet')

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="zh_CN")
tokenizer_en = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX")

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class Generalization:

    def __init__(self,path,save_path):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        for i, sent in enumerate(text):
            if i%2 == 0:
                self.en_li.append(sent.strip())
            elif i%2 == 1:
                self.zh_li.append(sent.strip())
        self.save_path = save_path

    def idiom(self):
        e = 1
        for en,zh in zip(self.en_li, self.zh_li):
            doc = nlp_en(sent)
            en_idiom = idiomatcher.identify(doc)
            if en_idiom != []:
                print(en)
                print(zh)
                print(en_idiom)


    def pronoun(self):
        r = 1
        for en,zh in zip(self.en_li, self.zh_li):
            if '他们' in zh and 'hey' not in en and 'heir' not in en and 'them' not in en and 'who' not in en and 'that' not in en: #and '所有' not in zh and '全部' not in zh and '全' not in zh and '都' not in zh and '一' not in zh and 'at all' not in en: #and "it's" not in en and "it comes" not in en and "that" not in en:
                # if en_tok_li.index('it') != 0 and "".join(tok.pos_ for tok in nlp_en(en_tok_li[en_tok_li.index('it')-1])) in ['VERB','ADV']:
                    
                seg_zh = posseg.cut(zh)
                noun_list = [tok.text for tok in nlp_en(en) if tok.pos_ in ['NOUN'] and tok.text != tok.lemma_]
                if noun_list != []:
                    print(en)
                    print(zh)
                    print(noun_list)
                    print()
                    with open(save_path,'a+') as q:
                        q.write(en+'\n'+zh+'\n'+str(noun_list)+'\n'+'他们:'+'\n')    ####这里要改！！！！！！！！！！！
                        print(r)
                        r += 1

    def hypernym(self):
        r = 1
        for en,zh in zip(self.en_li, self.zh_li):
            en_tok_li = [tok.text for tok in nlp_en(en)]
            zh_tok_li = jieba.cut(zh)
            zh2en_dict = {}
            for tok_zh in zh_tok_li:
                en_token = tokenizer(tok_zh, return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
                zh_text_trans = [tok.lemma_ for tok in nlp_en(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])][0]
                # print(zh_text_trans)
                zh2en_dict[zh_text_trans] = tok_zh
            
            print(r)
            r+=1
            for en_tok in en_tok_li:
                
                if wordnet.synsets(en_tok) != []:
                    # print(en_tok)
                    hypernyms = wordnet.synsets(en_tok)[0].hypernyms()
                    hyper_li = [h.lemma_names() for h in hypernyms]
                    if hyper_li != []:
                        hyper_li_final = []
                        hyper_str = '_'.join(hyper_li[0])
                        for ph in hyper_li[0]:
                            if '_' in ph:
                                new_ph = ph.replace('_',' ')
                                hyper_li_final.append(new_ph)
                            else:
                                hyper_li_final.append(ph)
                                
                    
                        for en2zh in zh2en_dict.keys():
                            if en2zh in hyper_li_final:
                                print(en)
                                print(zh)
                                print(en_tok, zh2en_dict[en2zh])
                                print()
                                with open(save_path,'a+') as q:
                                    q.write(en+'\n'+zh+'\n'+en_tok+':'+zh2en_dict[en2zh]+'\n')



    def hyponym(self):
        r = 1
        for en,zh in zip(self.en_li[45000:], self.zh_li[45000:]):
            en_tok_li = [tok.text for tok in nlp_en(en)]
            zh_tok_li = jieba.cut(zh)
            zh2en_dict = {}
            for tok_zh in zh_tok_li:
                en_token = tokenizer(tok_zh, return_tensors="pt")
                generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
                zh_text_trans = [tok.lemma_ for tok in nlp_en(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])][0]
                # print(zh_text_trans)
                zh2en_dict[zh_text_trans] = tok_zh
            
            print(r)
            r+=1
            for en_tok in en_tok_li:
                print(en_tok)
                
                if wordnet.synsets(en_tok) != []:
                    # print(en_tok)
                    hyponyms = wordnet.synsets(en_tok)[0].hyponyms()
                    hypo_li = [h.lemma_names() for h in hyponyms]
                    if hypo_li != []:
                        hypo_li_final = []
                        for ph in hypo_li[0]:
                            if '_' in ph:
                                new_ph = ph.replace('_',' ')
                                hypo_li_final.append(new_ph)
                            else:
                                hypo_li_final.append(ph)
                        print(hypo_li_final)
                                
                    
                        for en2zh in zh2en_dict.keys():
                            if en2zh in hypo_li_final:
                                print(en)
                                print(zh)
                                print(en_tok, zh2en_dict[en2zh])
                                print()
                                with open(save_path,'a+') as q:
                                    q.write(en+'\n'+zh+'\n'+en_tok+':'+zh2en_dict[en2zh]+'\n')







spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/Bilingual/Subtitles/Bi-Subtitles.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/hyponym_subtitles_new.txt'             
        
ptcl = Generalization(spath, save_path)

ptcl.hyponym()

In [6]:
class Cleaning:
    def __init__(self,path,save_path,n):
        with open(path) as f:
          text = f.readlines()
        self.en_li = []
        self.zh_li = []
        self.align_li = []
        self.n = n
        for i, sent in enumerate(text):
            if i%self.n == 0:
                self.en_li.append(sent.strip())
            elif i%self.n == 1:
                self.zh_li.append(sent.strip())
            elif i%self.n == 2:
                self.align_li.append(sent.strip())
        self.save_path = save_path

    def clean_duplicate(self):

        for en,zh,align_pair in zip(self.en_li, self.zh_li, self.align_li):
            en_part = align_pair.split(':')[0]
            zh_part = align_pair.split(':')[-1]

            zh_tok_li = jieba.cut(zh)
            zh_tok_li_rest = [zh_tok for zh_tok in zh_tok_li if zh_tok != zh_part]

            en_token = tokenizer_en(en_part, return_tensors="pt")
            generated_tokens = model.generate(**en_token, forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
            zh_text_trans = tokenizer_en.batch_decode(generated_tokens, skip_special_tokens=True)[0]
            # print(en_part)
            # print(zh_text_trans)

            if len(zh_part) != 0 and len(zh_text_trans) != 0:
                score = synonyms.compare(zh_part,zh_text_trans,seg=True)
                if score >=0.7:
                    real_align_li = []
                    for zh_tok_r in zh_tok_li_rest:

                        if len(zh_text_trans) !=0 and len(zh_tok_r) != 0:
                            r = synonyms.compare(zh_text_trans,zh_tok_r,seg=True)
                            if zh_text_trans not in zh and r <= 0.7 and len(zh_part)>1:
                                real_align = en_part+':'+zh_part
                                real_align_li.append(real_align)
                    for real in list(set(real_align_li)):
                        print(en)
                        print(zh)
                        print(real)
                        with open(self.save_path,'a+') as y:
                          y.write(en+'\n'+zh+'\n'+real+'\n')

                        





                


spath = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/to_be_clean_generalization/Laws.txt'
save_path = '/content/gdrive/MyDrive/Colab Notebooks/literal translationese/dataset/feature45_particular_generalization/to_be_clean_generalization/Laws_new.txt'             
        
ptcl = Cleaning(spath, save_path,3)

ptcl.clean_duplicate()


/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLIS is an electronic database of the legislation of Hong Kong. It is established and updated by the Department of Justice.
双语法例资料系统载有香港法例的电子版本，由律政司设立及更新。
database:资料


/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum le

viewing of legislation either on a provision-by-provision basis, or whole enactment basis.
单条条文模式及法例全文模式，让使用者以不同方式阅览法例。
enactment:法例


/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a,

...

(2) The audit under subsection (1) shall include an examination of overdue debts, if any, and a valuation of the assets and liabilities of the registered society.
(2) 第(1)款所指的审计须包括任何过期未还债项(如有的话)的审查及该注册合作社的资产与负债的估值。
debts:负债


/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = 

(b) appoint a day by notice before which creditors whose claims are not already recorded in the books of the society shall state their claims for admission or to be excluded from any distribution made before they have proved them;
(b) 藉公告指定一日期，规定在该日期前，凡有申索尚未记录於合作社簿册内的债权人，须就其申索作出陈述，以期获得接纳，债权人在证明其申索之前，不会获合作社的任何派发;
notice:公告
(1) In the liquidation of a society whose registration has been cancelled, the funds, including the reserve fund, shall be applied first to the costs of liquidation, then to the discharge of the liabilities of the society, then to the payment of the share capital and then, provided the by-laws of the society permit, to the payment of a dividend at a rate not exceeding ten per cent per annum for any period for which no disposal of profits was made.
(1) 任何被取消注册的合作社进行清盘时，其资金(包括储备金)须首先运用於支付清盘费用，其後用於清偿合作社的债务，然後再用於支付股本，如该合作社的章程许可而又有某段期间并无将利润作出处置，则余下资金须用於支付该段期间的股息，但年息不得超过10厘。
dividend:利润
(2) When the liquidation of a society has been closed and any creditor of that society h

/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in double_scalars
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in double_scalars
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
/usr/local/lib/python3.10/dist-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lam

(1) Unless otherwise authorized by the Governor under the proviso to subsection (1) of section 34 of the Ordinance, no dividend or payment on account of profits shall be made by a society registered with unlimited liability until the reserve fund has reached a proportion of not less than one tenth of the society's total liabilities.
(1) 除非总督根据本条例第34条第(1)款下的但书另行授权，否则以无限法律责任注册的合作社须待储备金达到不少於合作社负债总额十分之一的比例，始能就利润派发股息或款项。
dividend:利润
(2) The maximum so fixed shall be subject to the sanction of the Registrar, who may at any time reduce it. No registered society shall receive loans or deposits which will make the total amount of its debts exceed the limit sanctioned by the Registrar.
(2) 如此订定的最高债项总额须获注册官核准，并可由注册官随时减低。注册合作社不得接受会使其债项总额超过注册官核准限额的贷款或存款。
reduce:减低
